# Your First Click Model: Click Through Rate

This section examines the session data and computes the probability of relevance using Click-Through-Rate. Roughly the number of clicks divided by the number of sessions. Then we examine wheter there's position bias in that data - that is, consider perhaps that some documents have a higher CTR only because they show up higher in the search results.

In [1]:
import sys
sys.path.append("..")
sys.path.append("../ltr")
from aips import *
import matplotlib.pyplot as plt
import numpy
import pandas 
import random

from ltr.sdbn_functions import *

# if using a Jupyter notebook, includue:
%matplotlib inline

In [2]:
all_sessions()

sess_id                          query  rank        doc_id  clicked
0        50002                       blue ray   0.0  600603141003     True
1        50002                       blue ray   1.0  827396513927    False
2        50002                       blue ray   2.0   24543672067    False
3        50002                       blue ray   3.0  719192580374    False
4        50002                       blue ray   4.0  885170033412     True
...        ...                            ...   ...           ...      ...
74995     5001  transformers dark of the moon  10.0   47875841369    False
74996     5001  transformers dark of the moon  11.0   97363560449    False
74997     5001  transformers dark of the moon  12.0   93624956037    False
74998     5001  transformers dark of the moon  13.0   97363532149    False
74999     5001  transformers dark of the moon  14.0  400192926087    False

[1775000 rows x 5 columns]

In [3]:
sessions = all_sessions()
products = fetch_products(doc_ids=sessions["doc_id"].unique())

# Listing 11.03

Viewing session 2 of query `transformers dark of the moon` in retrotech. Here we inspect one of the sessions. We encourage you to examine other sessions

In [4]:
query = "transformers dark of the moon"
sessions = get_sessions(query)
sessions.loc[3]

query  rank        doc_id  clicked
sess_id                                                            
3        transformers dark of the moon   0.0   47875842328    False
3        transformers dark of the moon   1.0   24543701538    False
3        transformers dark of the moon   2.0   25192107191    False
3        transformers dark of the moon   3.0   47875841420    False
3        transformers dark of the moon   4.0  786936817218    False
3        transformers dark of the moon   5.0   47875842335    False
3        transformers dark of the moon   6.0   97363532149    False
3        transformers dark of the moon   7.0   97360810042     True
3        transformers dark of the moon   8.0   24543750949    False
3        transformers dark of the moon   9.0   36725235564    False
3        transformers dark of the moon  10.0   47875841369    False
3        transformers dark of the moon  11.0   97363560449    False
3        transformers dark of the moon  12.0   93624956037    False
3        transformers dark of the moon  13.0   47875841406    False
3        transformers dark of the moon  14.0  400192926087    False

# Listing 11.04

Simple CTR based judgments for our query. We compute the CTR by taking the number of clicks for a document relative to the number of unique sessions the doc appears in for that query.

In [5]:
#%load -s calculate_ctr ../ltr/sdbn_functions.py
def calculate_ctr(sessions):
    click_counts = sessions.groupby("doc_id")["clicked"].sum()
    sess_counts = sessions.groupby("doc_id")["sess_id"].nunique()
    ctrs = click_counts / sess_counts
    return ctrs.sort_values(ascending=False)

In [6]:
query = "transformers dark of the moon"
sessions = get_sessions(query, index=False)
ctrs = calculate_ctr(sessions)
ctrs

doc_id
97360810042     0.0824
47875842328     0.0734
47875841420     0.0434
24543701538     0.0364
25192107191     0.0352
786936817218    0.0236
97363560449     0.0192
47875841406     0.0160
400192926087    0.0124
47875842335     0.0106
97363532149     0.0084
36725235564     0.0082
93624956037     0.0082
47875841369     0.0074
24543750949     0.0062
dtype: float64

# Figure 11.2

Source code to render CTR judgment's ideal relevance ranking for `transformers dark of the moon`. In other words, our search results ordered from highest CTR to lowest.

In [7]:
render_judged(products,
              ctrs.to_frame(name="ctr").sort_values("ctr", ascending=False),
              grade_col="ctr",
              label=f"Click-Thru-Rate Judgments for q={query}")

# Figure 11.3

Source code to render CTR ideal relevance ranking for `dryer`. Ordering the highest CTR result to the lowest.

In [8]:
sessions = get_sessions("dryer", index=False)
ctrs = calculate_ctr(sessions)

render_judged(products,
              ctrs.to_frame(name="ctr").sort_values("ctr", ascending=False),
              grade_col="ctr",
              label=f"Click-Thru-Rate Judgments for q={query}")

## Listing 11.05

Computing the global CTR of each rank per search ranking to consider whether the click data is biased by position. We look over every search to see what the CTR is when a document is placed in a specific rank.

In [9]:
sessions = all_sessions()
num_sessions = len(sessions["sess_id"].unique())
global_ctrs = sessions.groupby("rank")["clicked"].sum() / num_sessions
global_ctrs

rank
0.0     0.249727
1.0     0.142673
2.0     0.084218
3.0     0.063073
4.0     0.056255
5.0     0.042255
6.0     0.033236
7.0     0.038000
8.0     0.020964
9.0     0.017364
10.0    0.013982
11.0    0.018582
12.0    0.015982
13.0    0.014509
14.0    0.012327
15.0    0.010200
16.0    0.011782
17.0    0.007891
18.0    0.007273
19.0    0.008145
20.0    0.006236
21.0    0.004473
22.0    0.005455
23.0    0.004982
24.0    0.005309
25.0    0.004364
26.0    0.005055
27.0    0.004691
28.0    0.005000
29.0    0.005400
Name: clicked, dtype: float64

## Listing 11.06

We look at the documents for our query, and notice that certain ones tend to appear higher and others tend to appear lower. If irrelevant ones dominate the top listings, position bias will dominate our training data

In [10]:
# %load -s calculate_average_rank ../ltr/sdbn_functions.py
def calculate_average_rank(sessions):
    avg_rank = sessions.groupby("doc_id")["rank"].mean()
    return avg_rank.sort_values(ascending=True)

In [11]:
sessions = get_sessions("transformers dark of the moon")
avg_rank = calculate_average_rank(sessions)
avg_rank

doc_id
47875842328      0.9808
24543701538      1.8626
25192107191      2.6596
47875841420      3.5344
786936817218     4.4444
47875842335      5.2776
47875841406      6.1378
97360810042      7.0130
24543750949      7.8626
36725235564      8.6854
47875841369      9.5796
97363560449     10.4304
93624956037     11.3298
97363532149     12.1494
400192926087    13.0526
Name: rank, dtype: float64

# Figure 11.4

In [12]:
render_judged(products, 
              avg_rank.reset_index().sort_values("rank"),
              grade_col="rank",
              label=f"Typical Search Session for q={query}")

In [13]:
query = "dryer"
sessions = get_sessions(query)
avg_rank = calculate_average_rank(sessions)
avg_rank

doc_id
12505451713      1.9124
84691226727      2.8290
883049066905     3.5726
84691226703      4.4552
74108056764      5.1276
77283045400      5.9318
783722274422     6.7610
665331101927     7.5138
14381196320      8.3308
74108096487      9.1230
36172950027      9.8758
12505525766     10.6500
48231011402     11.4734
74108007469     12.2940
883929085118    12.9996
36725561977     13.8932
36725578241     14.7286
12505527456     15.3526
48231011396     16.1548
856751002097    17.0208
Name: rank, dtype: float64

In [14]:
render_judged(products, 
              avg_rank.reset_index().sort_values("rank"),
              grade_col="rank",
              label=f"Typical Search Session for q={query}")

Up next: [Using SDBN Click Model To Overcome Position Bias](2.SDBN-judgments-to-overcome-posn-bias.ipynb)